Walt Disney Pictures films

Import libraries

In [25]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
import re

function which extracts the movie info from all the movie list

In [60]:
def movie_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(" ",strip=True).replace(u'\xa0',u' ') for li in row_data.find_all('li')]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ",strip=True).replace('\xa0',' ')

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()

def get_info_box(url):
    page = req.get(url)    
    soup = bs(page.content,'html.parser')
    clean_tags(soup)
    info_box = soup.find(class_ = 'infobox vevent')
    box_rows = info_box.find_all('tr')

    movie_info = {}
    for index,row in enumerate(box_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(" ",strip=True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find('th').get_text(" ",strip=True)
                content_value = movie_content_value(row.find('td'))
                movie_info[content_key] = content_value
    return movie_info

performing extraction part

In [61]:
url = 'https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films'
page = req.get(url)    
soup = bs(page.content,'html.parser')
movies = soup.select('.wikitable.sortable i a')
movie_info_list = []
# movie_links_list = [i.a for i in soup.table.find_all('i')]
for index,movie in enumerate(movies):
    try:
        link = "https://en.wikipedia.org" + movie['href']
        title = movie['title']
        movie_info_list.append(get_info_box(link))
    except Exception as e:
        print(e)
    print("Moive " + str(index) + " successfully scrapped")

Moive 0 successfully scrapped
Moive 1 successfully scrapped
Moive 2 successfully scrapped
Moive 3 successfully scrapped
Moive 4 successfully scrapped
Moive 5 successfully scrapped
Moive 6 successfully scrapped
Moive 7 successfully scrapped
Moive 8 successfully scrapped
Moive 9 successfully scrapped
Moive 10 successfully scrapped
Moive 11 successfully scrapped
Moive 12 successfully scrapped
Moive 13 successfully scrapped
Moive 14 successfully scrapped
Moive 15 successfully scrapped
Moive 16 successfully scrapped
Moive 17 successfully scrapped
Moive 18 successfully scrapped
Moive 19 successfully scrapped
Moive 20 successfully scrapped
Moive 21 successfully scrapped
Moive 22 successfully scrapped
Moive 23 successfully scrapped
Moive 24 successfully scrapped
Moive 25 successfully scrapped
Moive 26 successfully scrapped
Moive 27 successfully scrapped
Moive 28 successfully scrapped
Moive 29 successfully scrapped
Moive 30 successfully scrapped
Moive 31 successfully scrapped
Moive 32 successfu

In [62]:
len(movie_info_list)

521

saving and loading json file

In [63]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [64]:
import json

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [66]:
save_data("disney_data_cleaned.json", movie_info_list)

cleaning the data

In [102]:
movie_info_list = load_data('disney_data_cleaned.json')

sub tasks:
    cleaning references [1] (Done) ---
    convert running time into integer(Done) -----
    convert dates into datetime object ----
    split up long strings(Done) -----
    convert budget boxoffice into numbers (Done).

cleaning references []

In [103]:
def minutes_to_int(running_time):
    if running_time == 'N/A':
        return None
    elif isinstance(running_time,list):
        return int(running_time[0].split()[0])
    else:
        return int(running_time.split()[0])

for movie in movie_info_list:
    movie['Running time in hrs'] = minutes_to_int(movie.get('Running time','N/A'))


In [109]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

'''
money_conversion("$12.2 million") --> 12200000 ## Word syntax
money_conversion("$790,000") --> 790000        ## Value syntax
'''
def money_conversion(money):
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]
        
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

    else:
        return None

In [116]:
for index,movie in enumerate(movie_info_list):
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))
    print('converted at '+str(index)+' successfully')

converted at 0 successfully
converted at 1 successfully
converted at 2 successfully
converted at 3 successfully
converted at 4 successfully
converted at 5 successfully
converted at 6 successfully
converted at 7 successfully
converted at 8 successfully
converted at 9 successfully
converted at 10 successfully
converted at 11 successfully
converted at 12 successfully
converted at 13 successfully
converted at 14 successfully
converted at 15 successfully
converted at 16 successfully
converted at 17 successfully
converted at 18 successfully
converted at 19 successfully
converted at 20 successfully
converted at 21 successfully
converted at 22 successfully
converted at 23 successfully
converted at 24 successfully
converted at 25 successfully
converted at 26 successfully
converted at 27 successfully
converted at 28 successfully
converted at 29 successfully
converted at 30 successfully
converted at 31 successfully
converted at 32 successfully
converted at 33 successfully
converted at 34 successf

datetime object

In [132]:
from datetime import datetime

dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
        
    if date == "N/A":
        return None
        
    date_str = clean_date(date)

    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

In [133]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [141]:
import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [143]:
import pickle

def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [142]:
save_data_pickle("disney_movie_data_cleaned_more.pickle", movie_info_list)

In [144]:
a = load_data_pickle("disney_movie_data_cleaned_more.pickle")

In [145]:
a == movie_info_list

True

In [146]:
movie_info_list = load_data_pickle('disney_movie_data_cleaned_more.pickle')

In [153]:
movie_info_list[66]

{'title': 'The Incredible Journey',
 'Directed by': 'Fletcher Markle',
 'Written by': 'James Algar',
 'Based on': ['The Incredible Journey', 'by', 'Sheila Burnford'],
 'Produced by': ['James Algar', 'Walt Disney'],
 'Starring': ['Syn',
  'Émile Genest',
  'John Drainie',
  'Tommy Tweed',
  'Sandra Scott'],
 'Narrated by': 'Rex Allen',
 'Cinematography': 'Kenneth Peach',
 'Edited by': 'Norman R. Palmer',
 'Music by': 'Oliver Wallace',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': 'November 20, 1963',
 'Running time': '80 min.',
 'Countries': ['United States', 'Canada'],
 'Language': 'English',
 'Box office': '$4.2 million (US/Canada) (rentals)',
 'Running time in hrs': 80,
 'Budget (float)': None,
 'Box office (float)': 4200000.0,
 'Release date (datetime)': datetime.datetime(1963, 11, 20, 0, 0)}

In [154]:
movie_info_copy = [movie.copy() for movie in movie_info_list]

In [155]:
for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [156]:
movie_info_copy[3]

{'title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running time in hrs': 126,
 'Budget (float)': 2280000.0,
 'Box office (float)': 83300000.0,
 'Release date (datetime)': 'November 13, 1940'}

In [157]:
save_data("disney_data_final.json", movie_info_copy)


In [158]:
import pandas as pd

df = pd.DataFrame(movie_info_list)
df.to_csv("disney_movie_data_final.csv")

In [160]:
dff = pd.read_csv('disney_movie_data_final.csv')
dff.head()

,Unnamed: 0,title,Production company,Distributed by,Release date,Running time,Country,Language,Box office,Running time in hrs,...,Original concept by,Music,Lyrics,Book,Basis,Productions,Awards,Created by,Original work,Owner
0,0,Academy Award Review of,Walt Disney Productions,United Artists,"['May 19, 1937']",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Snow White and the Seven Dwarfs,Walt Disney Productions,RKO Radio Pictures,NaN,83 minutes,United States,English,$418 million,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Pinocchio,Walt Disney Productions,RKO Radio Pictures,NaN,88 minutes,United States,English,$164 million,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Fantasia,Walt Disney Productions,RKO Radio Pictures,"['November 13, 1940']",126 minutes,United States,English,$76.4–$83.3 million (United States and Canada),126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,The Reluctant Dragon,Walt Disney Productions,RKO Radio Pictures,"['June 27, 1941']",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
